In [1]:
import pandas as pd
import re
import emoji
from konlpy.tag import Okt

import os
import openai
import time

In [ ]:
# # 29주차 데이터 & 07.18 데이터 활용
# not_conn_media = pd.read_parquet("C:/Users/ehddl/Downloads/merged_data (2).parquet")
# conn_media = pd.read_parquet("C:/Users/ehddl/Downloads/merged_data (3).parquet")

In [ ]:
# recent_media_data = pd.concat([not_conn_media, conn_media], axis=0)
# recent_media_data.to_csv("recent_media_data.csv")

In [ ]:
# # 29주차 profile_data & 07.18 데이터 활용
# not_conn = pd.read_parquet("C:/Users/ehddl/Downloads/merged_data (4).parquet")
# conn = pd.read_parquet("C:/Users/ehddl/Downloads/merged_data (5).parquet")

In [ ]:
# recent_user_data = pd.concat([not_conn, conn], axis=0)
# recent_user_data.to_csv("recent_user_data.csv")

In [44]:
recent_media_data = pd.read_csv("recent_media_data.csv")
recent_user_data = pd.read_csv("recent_user_data.csv")

In [45]:
merged_data = pd.merge(recent_media_data, recent_user_data, on='acnt_id', how='left')
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41459 entries, 0 to 41458
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0_x            41459 non-null  int64  
 1   acnt_id                 41459 non-null  int64  
 2   media_id                41459 non-null  int64  
 3   media_type_nm           41459 non-null  object 
 4   reels_feed_type_nm      41459 non-null  object 
 5   media_url_addr          41418 non-null  object 
 6   media_unq_url_addr      41459 non-null  object 
 7   tmnl_url_addr           14686 non-null  object 
 8   reg_dt                  41459 non-null  object 
 9   media_cn                40784 non-null  object 
 10  acnt_conn_yn_x          41459 non-null  object 
 11  feed_share_yn           109 non-null    object 
 12  cmnt_actvtn_yn          323 non-null    object 
 13  Unnamed: 0_y            41194 non-null  float64
 14  acnt_nm                 41194 non-null

In [46]:
content = merged_data[['acnt_id', 'acnt_nm', 'acnt_sub_nm', 'intro_txt', 'media_id', 'media_type_nm', 'media_cn']]
content

,acnt_id,acnt_nm,acnt_sub_nm,intro_txt,media_id,media_type_nm,media_cn
0,17841400025364370,9_5jw,지원쓰 ( 3 10 1),⚡️,17992140674134243,IMAGE,휴가 돌려죠🫥
1,17841400025364370,9_5jw,지원쓰 ( 3 10 1),⚡️,17994028021438411,VIDEO,관종들 \n\n#릴스 #릴스타그램 #릴스초보
2,17841400025364370,9_5jw,지원쓰 ( 3 10 1),⚡️,18156434299170999,IMAGE,"날이 좋아서,"
3,17841400025364370,9_5jw,지원쓰 ( 3 10 1),⚡️,18313778404100377,CAROUSEL_ALBUM,"행복했던 9월, 고마워💚"
4,17841400039750009,bichon.mayo,뚱마요 mayo,"▪️11짤. 주세요 만렙, 개인기 다양함\n▪️네이버 펫인플루언서 @뚱마요\n▪️엄...",18029693123289447,VIDEO,#협찬 \n\n✔︎ 동결건조야채블럭 1개로 13종의 보라야채와 유산균 섭취가능\n✔...
...,...,...,...,...,...,...,...
41454,17841453615191128,bong_camper83,"봉캠퍼 | 잘봐요, 매번 신선한지[캠핑&캠핑용품 리뷰]",[좌충우돌 2인조 캠핑크루 J&B]\n@jamesjun_camp & @bong_ca...,18397690807128147,VIDEO,"아빠들의 일사분란함은 가족애 였다..\n\n어제 캠핑하는데,\n전혀 예상치 못했던 ..."
41455,17841453615191128,bong_camper83,"봉캠퍼 | 잘봐요, 매번 신선한지[캠핑&캠핑용품 리뷰]",[좌충우돌 2인조 캠핑크루 J&B]\n@jamesjun_camp & @bong_ca...,18398697055106070,VIDEO,#광고| 🍳식용유 없이 전을 굽는다고?\n\n풀무원 철판수제전 3종세트👇\n🥔철판 ...
41456,17841453615191128,bong_camper83,"봉캠퍼 | 잘봐요, 매번 신선한지[캠핑&캠핑용품 리뷰]",[좌충우돌 2인조 캠핑크루 J&B]\n@jamesjun_camp & @bong_ca...,18468362203078297,VIDEO,"제품제공 | 이걸 소개할 수 있어 영광입니다. 🙏\n\n✔️가장 카고스럽게, \n✔..."
41457,17841453615191128,bong_camper83,"봉캠퍼 | 잘봐요, 매번 신선한지[캠핑&캠핑용품 리뷰]",[좌충우돌 2인조 캠핑크루 J&B]\n@jamesjun_camp & @bong_ca...,18487771105065664,VIDEO,제품제공| 이제 남편들이 요리 다하겠습니다!🤙\n\n아내님들 남편에게 식스볼트 듀라...


In [47]:
print(len(content.drop_duplicates(subset='media_cn')))
print(len(content.dropna(subset='media_cn')))

40423
40784


In [48]:
content = content.dropna(subset='media_cn')
content = content.drop_duplicates(subset='media_cn')

In [49]:
len(content)

40422

In [53]:
def clean_text(text):
    if not isinstance(text, str):
        return ''
    
    text = emoji.replace_emoji(text, replace='')
    text = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', text)
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)

    return text.strip()

In [54]:
content['acnt_sub_nm_cleaned'] = content['acnt_sub_nm'].apply(clean_text)
content['intro_txt_cleaned'] = content['intro_txt'].apply(clean_text)
content['media_cn_cleaned'] = content['media_cn'].apply(clean_text)

In [55]:
content

,acnt_id,acnt_nm,acnt_sub_nm,intro_txt,media_id,media_type_nm,media_cn,acnt_sub_nm_cleaned,intro_txt_cleaned,media_cn_cleaned
0,17841400025364370,9_5jw,지원쓰 ( 3 10 1),⚡️,17992140674134243,IMAGE,휴가 돌려죠🫥,3 10 1,,휴가 돌려죠
1,17841400025364370,9_5jw,지원쓰 ( 3 10 1),⚡️,17994028021438411,VIDEO,관종들 \n\n#릴스 #릴스타그램 #릴스초보,3 10 1,,관종들 릴스 릴스타그램 릴스초보
2,17841400025364370,9_5jw,지원쓰 ( 3 10 1),⚡️,18156434299170999,IMAGE,"날이 좋아서,",3 10 1,,날이 좋아서
3,17841400025364370,9_5jw,지원쓰 ( 3 10 1),⚡️,18313778404100377,CAROUSEL_ALBUM,"행복했던 9월, 고마워💚",3 10 1,,행복했던 9월 고마워
4,17841400039750009,bichon.mayo,뚱마요 mayo,"▪️11짤. 주세요 만렙, 개인기 다양함\n▪️네이버 펫인플루언서 @뚱마요\n▪️엄...",18029693123289447,VIDEO,#협찬 \n\n✔︎ 동결건조야채블럭 1개로 13종의 보라야채와 유산균 섭취가능\n✔...,뚱마요 mayo,11짤 주세요 만렙 개인기 다양함 네이버 펫인플루언서 뚱마요 엄마는 kkf 애견미용...,협찬 동결건조야채블럭 1개로 13종의 보라야채와 유산균 섭취가능 보라색 안토시아닌이...
...,...,...,...,...,...,...,...,...,...,...
41454,17841453615191128,bong_camper83,"봉캠퍼 | 잘봐요, 매번 신선한지[캠핑&캠핑용품 리뷰]",[좌충우돌 2인조 캠핑크루 J&B]\n@jamesjun_camp & @bong_ca...,18397690807128147,VIDEO,"아빠들의 일사분란함은 가족애 였다..\n\n어제 캠핑하는데,\n전혀 예상치 못했던 ...",봉캠퍼 잘봐요 매번 신선한지캠핑캠핑용품 리뷰,좌충우돌 2인조 캠핑크루 jb jamesjuncamp bongcamper83 amb...,아빠들의 일사분란함은 가족애 였다 어제 캠핑하는데 전혀 예상치 못했던 비바람 돌풍이...
41455,17841453615191128,bong_camper83,"봉캠퍼 | 잘봐요, 매번 신선한지[캠핑&캠핑용품 리뷰]",[좌충우돌 2인조 캠핑크루 J&B]\n@jamesjun_camp & @bong_ca...,18398697055106070,VIDEO,#광고| 🍳식용유 없이 전을 굽는다고?\n\n풀무원 철판수제전 3종세트👇\n🥔철판 ...,봉캠퍼 잘봐요 매번 신선한지캠핑캠핑용품 리뷰,좌충우돌 2인조 캠핑크루 jb jamesjuncamp bongcamper83 amb...,광고 식용유 없이 전을 굽는다고 풀무원 철판수제전 3종세트 철판 바삭감자채전 철판 ...
41456,17841453615191128,bong_camper83,"봉캠퍼 | 잘봐요, 매번 신선한지[캠핑&캠핑용품 리뷰]",[좌충우돌 2인조 캠핑크루 J&B]\n@jamesjun_camp & @bong_ca...,18468362203078297,VIDEO,"제품제공 | 이걸 소개할 수 있어 영광입니다. 🙏\n\n✔️가장 카고스럽게, \n✔...",봉캠퍼 잘봐요 매번 신선한지캠핑캠핑용품 리뷰,좌충우돌 2인조 캠핑크루 jb jamesjuncamp bongcamper83 amb...,제품제공 이걸 소개할 수 있어 영광입니다 가장 카고스럽게 보다 더 대담하게 카고컨테...
41457,17841453615191128,bong_camper83,"봉캠퍼 | 잘봐요, 매번 신선한지[캠핑&캠핑용품 리뷰]",[좌충우돌 2인조 캠핑크루 J&B]\n@jamesjun_camp & @bong_ca...,18487771105065664,VIDEO,제품제공| 이제 남편들이 요리 다하겠습니다!🤙\n\n아내님들 남편에게 식스볼트 듀라...,봉캠퍼 잘봐요 매번 신선한지캠핑캠핑용품 리뷰,좌충우돌 2인조 캠핑크루 jb jamesjuncamp bongcamper83 amb...,제품제공 이제 남편들이 요리 다하겠습니다 아내님들 남편에게 식스볼트 듀라박스만 사주...


In [57]:
content[content['media_cn_cleaned'].str.contains('http')]

# url 값을 다 지워야 하려나..일단 삭제하지 말고 gpt한테 그거 참고해봐도 된다고 하자.

,acnt_id,acnt_nm,acnt_sub_nm,intro_txt,media_id,media_type_nm,media_cn,acnt_sub_nm_cleaned,intro_txt_cleaned,media_cn_cleaned
30,17841400194103434,elena_hyemini_ss,보혜미안🌿,"일상을 여행처럼, 하고픈 일하며 살래요✨\n🇰🇷✈️🇨🇳🇻🇳🇮🇩🇵🇭🇦🇺🇫🇷🇨🇭🇮🇹🇵🇹...",18036344756659440,CAROUSEL_ALBUM,#협찬 🤿\n시흥 거북섬 파라다이브에서 오랜만에 프리다이빙 체험!\n샤이프리다이빙 ...,보혜미안,일상을 여행처럼 하고픈 일하며 살래요,협찬 시흥 거북섬 파라다이브에서 오랜만에 프리다이빙 체험 샤이프리다이빙 윤석희 강사...
56,17841400507981948,honey_cloche,클로쉐〰️,11년차 Blog influencer \n▫[기능성 프리미엄 성수동 수제화 특허 구...,18071605745006709,CAROUSEL_ALBUM,#협찬 미숫가루 맛으로 맛있고 \n건강한 라인업 미식쉐이크 🥛\n\n미식쉐이크는 당...,클로쉐,11년차 blog influencer 기능성 프리미엄 성수동 수제화 특허 구두 cl...,협찬 미숫가루 맛으로 맛있고 건강한 라인업 미식쉐이크 미식쉐이크는 당류 1미만의 저...
63,17841400611875418,cafe.flex,카페플렉스 | 카페 좋아하는 사람들,☕️ 카페 좋아하는 사람들 카페플렉스\n🍰 100% 솔직 카페 후기만 모았어요.,17935427859008478,CAROUSEL_ALBUM,경기도 의정부 카페 추천 5곳\n\n1. 쇼트브레드\n라떼 한 잔에 분위기까지 더해...,카페플렉스 카페 좋아하는 사람들,카페 좋아하는 사람들 카페플렉스 100 솔직 카페 후기만 모았어요,경기도 의정부 카페 추천 5곳 1 쇼트브레드 라떼 한 잔에 분위기까지 더해지는 카페...
64,17841400611875418,cafe.flex,카페플렉스 | 카페 좋아하는 사람들,☕️ 카페 좋아하는 사람들 카페플렉스\n🍰 100% 솔직 카페 후기만 모았어요.,17959645844775715,CAROUSEL_ALBUM,경남 남해 카페 추천 5곳\n\n1. 고운동커피\n한줄평: 유채밭을 배경으로 커피 ...,카페플렉스 카페 좋아하는 사람들,카페 좋아하는 사람들 카페플렉스 100 솔직 카페 후기만 모았어요,경남 남해 카페 추천 5곳 1 고운동커피 한줄평 유채밭을 배경으로 커피 한잔 남해 ...
65,17841400611875418,cafe.flex,카페플렉스 | 카페 좋아하는 사람들,☕️ 카페 좋아하는 사람들 카페플렉스\n🍰 100% 솔직 카페 후기만 모았어요.,18089962297720366,CAROUSEL_ALBUM,경기도 남양주 카페 추천 5곳\n\n1. 후탄\n탁 트인 호수 뷰와 넓은 공간의 대...,카페플렉스 카페 좋아하는 사람들,카페 좋아하는 사람들 카페플렉스 100 솔직 카페 후기만 모았어요,경기도 남양주 카페 추천 5곳 1 후탄 탁 트인 호수 뷰와 넓은 공간의 대형 남양주...
...,...,...,...,...,...,...,...,...,...,...
41058,17841453155343189,ing.ing90,NaN,#🏕 #캠핑 #맛집 #블로거,18054939490804984,CAROUSEL_ALBUM,#내돈내산 \n\n고르고 골라 선택한 모모드 노엘 테이블!\n\n✔️그만큼 깐깐하게...,,캠핑 맛집 블로거,내돈내산 고르고 골라 선택한 모모드 노엘 테이블 그만큼 깐깐하게 고르고 골랐답니다 ...
41098,17841462888085401,mr___choon,춘시기,"👶🏻유이재 , 🦝유춘식 , 🐈‍⬛유깜식",17844407115533613,VIDEO,#광고 \n집이 늘 눅눅해서 걱정이었어요. \n춘시기도 힘없어 보이더라고요.\n\n...,춘시기,유이재 유춘식 유깜식,광고 집이 늘 눅눅해서 걱정이었어요 춘시기도 힘없어 보이더라고요 그래서 제습기를 하...
41122,17841467078397304,hyunnn138_,로지 ROSY,"🏥직장인 간호사\n🌷소소한 일상, 맛집, 여행, 뷰티, 패션, 리뷰\n🩷좋아하는것들...",17972589245841582,CAROUSEL_ALBUM,#협찬 업체로부터 제공받은 제품입니다 \n\n다이어터의 최대 고민 🥹 \n야채와 물...,로지 rosy,직장인 간호사 소소한 일상 맛집 여행 뷰티 패션 리뷰 좋아하는것들을 담아요 맞팔 디...,협찬 업체로부터 제공받은 제품입니다 다이어터의 최대 고민 야채와 물을 많이 먹는다고...
41279,17841449549923448,heihwi,휘원,ᴅᴀɪʟʏ / ᴡᴏʀᴋᴏᴜᴛ\n아르듀 5기 앰버서더 @ardu.me @ardu.we,17981834531803840,VIDEO,#광고\n인생 첫 메리제인은 비바이아로!\n재활용 플라스틱 병으로 만든 친환경 소재...,휘원,아르듀 5기 앰버서더 ardume arduwe,광고 인생 첫 메리제인은 비바이아로 재활용 플라스틱 병으로 만든 친환경 소재에 신축...


In [58]:
# final_content = content[['media_cn_preprocessed']]
# final_content

In [59]:
content.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40422 entries, 0 to 41458
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   acnt_id              40422 non-null  int64 
 1   acnt_nm              40172 non-null  object
 2   acnt_sub_nm          39143 non-null  object
 3   intro_txt            37501 non-null  object
 4   media_id             40422 non-null  int64 
 5   media_type_nm        40422 non-null  object
 6   media_cn             40422 non-null  object
 7   acnt_sub_nm_cleaned  40422 non-null  object
 8   intro_txt_cleaned    40422 non-null  object
 9   media_cn_cleaned     40422 non-null  object
dtypes: int64(2), object(8)
memory usage: 3.4+ MB


In [60]:
final_content = content[['acnt_sub_nm_cleaned', 'intro_txt_cleaned', 'media_cn_cleaned']]
final_content

,acnt_sub_nm_cleaned,intro_txt_cleaned,media_cn_cleaned
0,3 10 1,,휴가 돌려죠
1,3 10 1,,관종들 릴스 릴스타그램 릴스초보
2,3 10 1,,날이 좋아서
3,3 10 1,,행복했던 9월 고마워
4,뚱마요 mayo,11짤 주세요 만렙 개인기 다양함 네이버 펫인플루언서 뚱마요 엄마는 kkf 애견미용...,협찬 동결건조야채블럭 1개로 13종의 보라야채와 유산균 섭취가능 보라색 안토시아닌이...
...,...,...,...
41454,봉캠퍼 잘봐요 매번 신선한지캠핑캠핑용품 리뷰,좌충우돌 2인조 캠핑크루 jb jamesjuncamp bongcamper83 amb...,아빠들의 일사분란함은 가족애 였다 어제 캠핑하는데 전혀 예상치 못했던 비바람 돌풍이...
41455,봉캠퍼 잘봐요 매번 신선한지캠핑캠핑용품 리뷰,좌충우돌 2인조 캠핑크루 jb jamesjuncamp bongcamper83 amb...,광고 식용유 없이 전을 굽는다고 풀무원 철판수제전 3종세트 철판 바삭감자채전 철판 ...
41456,봉캠퍼 잘봐요 매번 신선한지캠핑캠핑용품 리뷰,좌충우돌 2인조 캠핑크루 jb jamesjuncamp bongcamper83 amb...,제품제공 이걸 소개할 수 있어 영광입니다 가장 카고스럽게 보다 더 대담하게 카고컨테...
41457,봉캠퍼 잘봐요 매번 신선한지캠핑캠핑용품 리뷰,좌충우돌 2인조 캠핑크루 jb jamesjuncamp bongcamper83 amb...,제품제공 이제 남편들이 요리 다하겠습니다 아내님들 남편에게 식스볼트 듀라박스만 사주...


category_labeling

In [ ]:
# os.environ['OPENAI_API_KEY'] = ''
# os.environ['MODEL_NAME'] = 'gpt-4o'

In [65]:
category_list = ['짤/밈', '시사', '엔터테인먼트', '셀럽', '반려동물', '미술/디자인', '베이비/키즈', '결혼/연애', '홈/리빙', '패션', '뷰티', '문구/완구', '다이어트/건강보조식품',
                 '스포츠', '취미', '게임', '사진/영상', '여행/관광', '푸드', '유명장소/핫플', '일상', '교육', '자동차/모빌리티', 'IT', '브랜드공식계정', '만화/애니/툰']

def generate_prompt(nickname, intro, content):
    return f"""
            너는 사용자의 SNS 프로필 정보(사용자 별명, 소개글), SNS 게시물 내용을 참고해서 아래 카테고리 중 가장 잘 맞는 카테고리를 최대 3개까지 선택하는 분류기야.
            카테고리 목록은 다음과 같아: {category_list}
            
            같은 사람이더라도 여러가지 카테고리를 가질 수 있잖아. 예를들어, 캠핑 관련된 글을 쓰는 사람이라면 '홈/리빙', '일상' 이렇게 두 가지를 가질수도 있는 거고 말야. 
            만약에 잘 모르겠으면 '기타'로 해주면 될 것 같아. 

            [사용자 정보]
            - 별명 : {nickname}
            - 소개글 : {intro}
            - 게시물: {content}
            
            이 사용자에게 적합한 카테고리를 최대 3개까지 한 줄에 '@'로 구분해서 출력해줘.

            """

def label_with_gpt(nickname, intro, content):
    import os
    import openai

    api_key = os.environ.get('OPENAI_API_KEY')
    model_name = os.environ.get('MODEL_NAME', 'gpt-4o')  # 기본값 추가
    
    if not api_key:
        return 'API Key가 설정되지 않았습니다.'

    openai.api_key = api_key

    prompt = generate_prompt(nickname, intro, content)

    try:
        response = openai.ChatCompletion.create(
            model=model_name,
            temperature=0,
            messages=[
                {"role": "system", "content": "너는 SNS 게시물 분류 전문가야. URL, 해시태그, 키워드를 잘 분석해서 적절한 카테고리를 고르고, 다중 분류도 잘해야 해."},
                {"role": "user", "content": prompt}
            ]
        )
        return response["choices"][0]["message"]["content"].strip()
    except Exception as e:
        print(f"Error: {e}")
        return "Unknown"


In [66]:
from tqdm import tqdm

# tqdm을 활용해 진행률 확인
results = []
for i, row in tqdm(final_content.iterrows(), total=len(final_content)):
    nickname = row['acnt_sub_nm_cleaned']
    intro = row['intro_txt_cleaned']
    post = row['media_cn_cleaned']
    
    label = label_with_gpt(nickname, intro, post)
    results.append(label)

final_content["gpt_labels"] = results

100%|██████████| 40422/40422 [8:52:11<00:00,  1.27it/s]   
C:\Users\ehddl\AppData\Local\Temp\ipykernel_14052\594970978.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_content["gpt_labels"] = results


In [73]:
final_content.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40422 entries, 0 to 41458
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   acnt_sub_nm_cleaned  40422 non-null  object
 1   intro_txt_cleaned    40422 non-null  object
 2   media_cn_cleaned     40422 non-null  object
 3   gpt_labels           40422 non-null  object
dtypes: object(4)
memory usage: 1.5+ MB


In [95]:
# final_content.loc[:, 'acnt_id'] = content['acnt_id']
# final_content.loc[:, 'acnt_nm'] = content['acnt_nm']
# final_content.loc[:, 'acnt_sub_nm'] = content['acnt_sub_nm']

In [70]:
# final_content.to_csv("1st_labelig_data.csv")

In [84]:
# 결과 포맷 후처리
def parse_labels(label_string):
    try:
        return [label.strip() for label in label_string.split('@') if label.strip()]
    except:
        return []

final_content["label_list"] = final_content["gpt_labels"].apply(parse_labels)

C:\Users\ehddl\AppData\Local\Temp\ipykernel_14052\1535174310.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_content["label_list"] = final_content["gpt_labels"].apply(parse_labels)


In [93]:
final_content['label'] = final_content['label_list'].apply(lambda x: x[0])
final_content.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40422 entries, 0 to 41458
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   acnt_sub_nm_cleaned  40422 non-null  object
 1   intro_txt_cleaned    40422 non-null  object
 2   media_cn_cleaned     40422 non-null  object
 3   gpt_labels           40422 non-null  object
 4   acnt_id              40422 non-null  int64 
 5   acnt_nm              40172 non-null  object
 6   acnt_sub_nm          39143 non-null  object
 7   label_list           40422 non-null  object
 8   label                40422 non-null  object
dtypes: int64(1), object(8)
memory usage: 3.1+ MB


C:\Users\ehddl\AppData\Local\Temp\ipykernel_14052\2277757666.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_content['label'] = final_content['label_list'].apply(lambda x: x[0])


In [99]:
final_content = final_content[['acnt_id', 'acnt_nm', 'acnt_sub_nm_cleaned', 'intro_txt_cleaned', 'media_cn_cleaned', 'label_list', 'label']]
final_content

,acnt_id,acnt_nm,acnt_sub_nm_cleaned,intro_txt_cleaned,media_cn_cleaned,label_list,label
0,17841400025364370,9_5jw,3 10 1,,휴가 돌려죠,[일상],일상
1,17841400025364370,9_5jw,3 10 1,,관종들 릴스 릴스타그램 릴스초보,"[짤/밈, 엔터테인먼트, 일상]",짤/밈
2,17841400025364370,9_5jw,3 10 1,,날이 좋아서,[일상],일상
3,17841400025364370,9_5jw,3 10 1,,행복했던 9월 고마워,[일상],일상
4,17841400039750009,bichon.mayo,뚱마요 mayo,11짤 주세요 만렙 개인기 다양함 네이버 펫인플루언서 뚱마요 엄마는 kkf 애견미용...,협찬 동결건조야채블럭 1개로 13종의 보라야채와 유산균 섭취가능 보라색 안토시아닌이...,"[반려동물, 푸드, 일상]",반려동물
...,...,...,...,...,...,...,...
41454,17841453615191128,bong_camper83,봉캠퍼 잘봐요 매번 신선한지캠핑캠핑용품 리뷰,좌충우돌 2인조 캠핑크루 jb jamesjuncamp bongcamper83 amb...,아빠들의 일사분란함은 가족애 였다 어제 캠핑하는데 전혀 예상치 못했던 비바람 돌풍이...,"[홈/리빙, 일상, 여행/관광]",홈/리빙
41455,17841453615191128,bong_camper83,봉캠퍼 잘봐요 매번 신선한지캠핑캠핑용품 리뷰,좌충우돌 2인조 캠핑크루 jb jamesjuncamp bongcamper83 amb...,광고 식용유 없이 전을 굽는다고 풀무원 철판수제전 3종세트 철판 바삭감자채전 철판 ...,"[홈/리빙, 일상, 푸드]",홈/리빙
41456,17841453615191128,bong_camper83,봉캠퍼 잘봐요 매번 신선한지캠핑캠핑용품 리뷰,좌충우돌 2인조 캠핑크루 jb jamesjuncamp bongcamper83 amb...,제품제공 이걸 소개할 수 있어 영광입니다 가장 카고스럽게 보다 더 대담하게 카고컨테...,"[홈/리빙, 일상, 여행/관광]",홈/리빙
41457,17841453615191128,bong_camper83,봉캠퍼 잘봐요 매번 신선한지캠핑캠핑용품 리뷰,좌충우돌 2인조 캠핑크루 jb jamesjuncamp bongcamper83 amb...,제품제공 이제 남편들이 요리 다하겠습니다 아내님들 남편에게 식스볼트 듀라박스만 사주...,"[홈/리빙, 일상, 취미]",홈/리빙


In [109]:
final_content.dropna(subset=['media_cn_cleaned'], inplace=True)

C:\Users\ehddl\AppData\Local\Temp\ipykernel_14052\484143670.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_content.dropna(subset=['media_cn_cleaned'], inplace=True)


In [113]:
# 라벨링 확인 작업
label_set = sorted(final_content['label'].unique())
label_id = {label : idx for idx, label in enumerate(label_set)}
label_id

{"'일상'": 0,
 "'취미": 1,
 "'취미'": 2,
 'IT': 3,
 '```\nIT': 4,
 '```\n브랜드공식계정': 5,
 '```\n스포츠': 6,
 '```\n여행/관광': 7,
 '```\n일상': 8,
 '게임': 9,
 '결혼/연애': 10,
 '경제': 11,
 '골린이': 12,
 '골프': 13,
 "골프 관련된 내용이므로 '스포츠'와 브랜드 관련 언급이 있어 '브랜드공식계정'이 적합해 보입니다.  \n스포츠": 14,
 '교육': 15,
 '국립현대미술관': 16,
 '기타': 17,
 '낚시': 18,
 "낚시 관련된 내용과 축제 정보를 공유하는 것으로 보아, 이 사용자는 '취미', '여행/관광', '일상' 카테고리에 적합할 것 같아. \n\n취미": 19,
 '다이어트/건강보조식품': 20,
 '러닝용품': 21,
 '만화/애니/툰': 22,
 '명품': 23,
 '문구/완구': 24,
 '문예/디자인': 25,
 '문학/책': 26,
 '미술/디자인': 27,
 '반려동물': 28,
 '베이비/키즈': 29,
 '부동산': 30,
 '뷰티': 31,
 '브랜드공식계정': 32,
 '사용자 정보가 제공되지 않았습니다. 사용자 별명, 소개글, 게시물 내용을 입력해주시면 적합한 카테고리를 분류해드리겠습니다.': 33,
 '사용자 정보가 제공되지 않았습니다. 사용자 별명, 소개글, 게시물 내용을 입력해주시면 적합한 카테고리를 선택해드리겠습니다.': 34,
 '사용자 정보가 제공되지 않았습니다. 사용자 정보를 입력해 주시면 적합한 카테고리를 분류해 드리겠습니다.': 35,
 '사용자 정보가 제공되지 않았습니다. 사용자 정보를 입력해주시면 적합한 카테고리를 분류해드리겠습니다.': 36,
 '사용자 정보가 제공되지 않았습니다. 사용자 정보를 입력해주시면 적합한 카테고리를 선택해드리겠습니다.': 37,
 '사진/영상': 38,
 '셀럽': 39,
 '스포츠': 40,
 '시사': 41,
 "아이폰 관련된 내용과 빈티지 아이폰에 대한

In [153]:
# 라벨링된 값 중 예시 Series (label 목록)
raw_labels = pd.Series([
    "```\\nIT", "'일상'", "취미", "'취미'", "정보가 부족하여 카테고리를 분류할 수 없습니다. '기타'로 분류합니다.",
    "프리다이빙과 관련된 활동... '스포츠'", "음악 관련 키워드... '엔터테인먼트'", 
    "죄송하지만, 사용자 정보가 제공되지 않았습니다.",
    "IT", "스포츠"
])

# 의미 없는 GPT 응답 키워드 목록
meaningless_phrases = [
    "사용자 정보가 제공되지 않았습니다",
    "죄송하지만",
    "정보가 부족하여",
    "입력해 주시면",
    "제공해주시면",
    "추천해 드리겠습니다",
    "분류할 수 없습니다",
    "주세요"
]

# 정제 함수
def clean_label(label):
    if pd.isna(label) or label.strip() == "":
        return "기타"
    
    # 1. 의미 없는 문장인 경우
    for phrase in meaningless_phrases:
        if phrase in label:
            return "기타"
    
    # 2. 라벨이 여러 줄/설명 포함된 경우는 마지막 줄 혹은 맨 마지막에 있는 한글 추출
    label = label.strip()
    
    # 3. 주석/백틱 제거
    label = re.sub(r"```|\\n", " ", label)
    
    # 4. 따옴표, 줄바꿈 등 제거
    label = label.strip("\"' \n\t")

    # 5. 가장 마지막 카테고리 추출 (한글 단어 기준)
    matches = re.findall(r"[가-힣/]+", label)
    if matches:
        return matches[-1]
    
    return "기타"


In [135]:
final_content['clean_label'] = final_content['label'].apply(clean_label)

In [136]:
# 라벨링 다시 확인
label_set = sorted(final_content['clean_label'].unique())
label_id = {label : idx for idx, label in enumerate(label_set)}
label_id

{'같습니다': 0,
 '같아': 1,
 '게임': 2,
 '결혼/연애': 3,
 '경남프리다이빙': 4,
 '경제': 5,
 '골린이': 6,
 '골프': 7,
 '교육': 8,
 '국립현대미술관': 9,
 '기타': 10,
 '낚시': 11,
 '다이어트/건강보조식품': 12,
 '러닝용품': 13,
 '만화/애니/툰': 14,
 '먹방유튜버': 15,
 '명품': 16,
 '문구/완구': 17,
 '문예/디자인': 18,
 '문학/책': 19,
 '미술/디자인': 20,
 '반려동물': 21,
 '베이비/키즈': 22,
 '부동산': 23,
 '뷰티': 24,
 '브랜드공식계정': 25,
 '사진/영상': 26,
 '셀럽': 27,
 '스포츠': 28,
 '시사': 29,
 '엔터테인먼트': 30,
 '여행/관광': 31,
 '유명장소/핫플': 32,
 '육아/키즈': 33,
 '음악/공연': 34,
 '음악/커버': 35,
 '일상': 36,
 '자동차/모빌리티': 37,
 '짤/밈': 38,
 '취미': 39,
 '타투': 40,
 '패션': 41,
 '퍼스널브랜딩': 42,
 '푸드': 43,
 '프리다이빙': 44,
 '홈/리빙': 45,
 '환경/지속가능성': 46,
 '환경보호': 47}

In [ ]:
# 남은거 몇 개만 수동 정제
category_mapping = {
    '프리다이빙': '스포츠',
    '골린이': '스포츠',
    '골프': '스포츠',
    '러닝용품': '스포츠',
    '낚시': '취미',
    '타투': '미술/디자인',
    '문예': '미술/디자인',
    '환경/지속가능성': '시사',
    '환경보호': '시사',
    '퍼스널브랜딩': '브랜드공식계정',
    '음악': '엔터테인먼트',
    '육아/키즈': '베이비/키즈',
    '명품': '패션',
    '문학/책': '시사',
    '국립현대미술관': '여행/관광',
    '먹방유튜버': '푸드',
    '부동산' : '기타',
    '경제': '교육'
}

# 각 키워드가 포함되면 해당 라벨로 교체
def relabel_category(row):
    for keyword, new_label in category_mapping.items():
        if keyword in row:
            return new_label
    return row  # 해당 없으면 원래 값 유지

final_content['clean_label'] = final_content['clean_label'].apply(relabel_category)

In [145]:
# lst = ['짤/밈', '시사', '엔터테인먼트', '셀럽', '반려동물', '미술/디자인', '베이비/키즈', '결혼/연애', '홈/리빙', '패션', '뷰티', '문구/완구', '다이어트/건강보조식품',
#  '스포츠', '취미', '게임', '사진/영상', '여행/관광', '푸드', '유명장소/핫플', '일상', '교육', '자동차/모빌리티', 'IT', '브랜드공식계정', '만화/애니/툰', '기타'
# ]
# len(lst)

27

In [139]:
final_content[final_content['clean_label'].str.contains("같")]
final_content.loc[24355, 'clean_label'] = 'IT'
final_content.loc[33218, 'clean_label'] = 'IT'

In [154]:
# 최종 라벨 확인
label_set = sorted(final_content['clean_label'].unique())
label_id = {label : idx for idx, label in enumerate(label_set)}
label_id

{'IT': 0,
 '게임': 1,
 '결혼/연애': 2,
 '교육': 3,
 '기타': 4,
 '다이어트/건강보조식품': 5,
 '만화/애니/툰': 6,
 '문구/완구': 7,
 '미술/디자인': 8,
 '반려동물': 9,
 '베이비/키즈': 10,
 '뷰티': 11,
 '브랜드공식계정': 12,
 '사진/영상': 13,
 '셀럽': 14,
 '스포츠': 15,
 '시사': 16,
 '엔터테인먼트': 17,
 '여행/관광': 18,
 '유명장소/핫플': 19,
 '일상': 20,
 '자동차/모빌리티': 21,
 '짤/밈': 22,
 '취미': 23,
 '패션': 24,
 '푸드': 25,
 '홈/리빙': 26}

In [ ]:
# final_content.to_csv("2nd_labeling_data.csv")

In [186]:
final_content['clean_label'].value_counts()

clean_label
여행/관광          6528
베이비/키즈         6086
뷰티             5950
푸드             4108
일상             3101
패션             2732
홈/리빙           2073
다이어트/건강보조식품    1859
반려동물           1510
기타             1155
엔터테인먼트          802
스포츠             714
브랜드공식계정         677
유명장소/핫플         669
미술/디자인          443
결혼/연애           423
교육              330
사진/영상           263
짤/밈             252
만화/애니/툰         180
자동차/모빌리티        174
셀럽              133
문구/완구           101
시사               75
게임               52
취미               30
IT                2
Name: count, dtype: int64

3rd Data preprocessing - labeling

수동 라벨링 확인 이후 label_id 값 얻기

In [226]:
final_content_2 = pd.read_csv("3rd_labeling_data.csv", index_col=0)

In [ ]:
# final_content_unique = final_content[['acnt_nm', 'acnt_id']].drop_duplicates(subset='acnt_nm')
# final_content_2 = pd.merge(final_content_2, final_content_unique, on='acnt_nm', how='left')

In [ ]:
# final_content_2.drop(['acnt_id_x'], axis=1, inplace=True)
# final_content_2.rename(columns={'acnt_id_y':'acnt_id'}, inplace=True)

In [ ]:
# final_content_2 = final_content_2[['acnt_id', 'acnt_nm', 'acnt_sub_nm_cleaned', 'intro_txt_cleaned', 'media_cn_cleaned', 'label_list', 'label', 'clean_label', 'revised_label']]

In [ ]:
# final_content_2.to_csv("3rd_labeling_data.csv")

In [228]:
fine_tuning_data = final_content_2.copy()

In [ ]:
# fine_tuning_data.drop(['acnt_sub_nm_cleaned', 'intro_txt_cleaned', 'label_list', 'label', 'clean_label'], axis=1, inplace=True)

In [240]:
fine_tuning_data = fine_tuning_data.drop_duplicates(subset=['acnt_id', 'acnt_nm', 'media_cn_cleaned'])

In [242]:
# 최종 라벨 확인 -> 기타를 전부 일상으로 뺐음
label_set = sorted(fine_tuning_data['revised_label'].unique())
label_id = {label : idx for idx, label in enumerate(label_set)}
label_id

{'IT': 0,
 '게임': 1,
 '결혼/연애': 2,
 '교육': 3,
 '다이어트/건강보조식품': 4,
 '만화/애니/툰': 5,
 '문구/완구': 6,
 '미술/디자인': 7,
 '반려동물': 8,
 '베이비/키즈': 9,
 '뷰티': 10,
 '브랜드공식계정': 11,
 '사진/영상': 12,
 '셀럽': 13,
 '스포츠': 14,
 '시사': 15,
 '엔터테인먼트': 16,
 '여행/관광': 17,
 '유명장소/핫플': 18,
 '일상': 19,
 '자동차/모빌리티': 20,
 '짤/밈': 21,
 '취미': 22,
 '패션': 23,
 '푸드': 24,
 '홈/리빙': 25}

In [243]:
fine_tuning_data['revised_label'].value_counts()

revised_label
베이비/키즈         5883
뷰티             5742
여행/관광          5339
유명장소/핫플        5112
일상             3660
패션             2746
홈/리빙           2052
푸드             1901
다이어트/건강보조식품    1812
반려동물           1458
엔터테인먼트          744
스포츠             700
미술/디자인          420
결혼/연애           397
IT              331
교육              331
사진/영상           251
짤/밈             246
만화/애니/툰         177
자동차/모빌리티        169
셀럽              133
문구/완구           100
시사               72
게임               51
취미               29
브랜드공식계정           7
Name: count, dtype: int64

In [249]:
id_label = {idx: label for label, idx in label_id.items()}
fine_tuning_data["label_id"] = fine_tuning_data["revised_label"].map(label_id)

C:\Users\ehddl\AppData\Local\Temp\ipykernel_14052\3163038952.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fine_tuning_data["label_id"] = fine_tuning_data["revised_label"].map(label_id)


In [ ]:
def is_blank(x):
    return pd.isna(x) or str(x).strip() == "" or str(x).strip() == " "

blank_rows = fine_tuning_data[
    fine_tuning_data["intro_txt_cleaned"].apply(is_blank) &
    fine_tuning_data["media_cn_cleaned"].apply(is_blank)]

print(len(blank_rows))

In [ ]:
fine_tuning_data = fine_tuning_data[
    ~(fine_tuning_data["intro_txt_cleaned"].apply(is_blank) &
      fine_tuning_data["media_cn_cleaned"].apply(is_blank))].reset_index(drop=True)

In [ ]:
# fine_tuning_data.to_csv("fine_tuning_data.csv")

In [ ]:
fine_tuning_data['length'] = fine_tuning_data['media_cn_cleaned'].apply(lambda x: len(str(x)))
fine_tuning_data = fine_tuning_data[fine_tuning_data['length'] > 6] # 공백포함+5글자로 해서 6글자 이상인 것 만 필터링

C:\Users\ehddl\AppData\Local\Temp\ipykernel_14052\3539248314.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fine_tuning_data['length'] = fine_tuning_data['media_cn_cleaned'].apply(lambda x: len(str(x)))


,acnt_id,acnt_nm,media_cn_cleaned,revised_label,label_id,length
1,17841400025364370,9_5jw,관종들 릴스 릴스타그램 릴스초보,짤/밈,21,17
3,17841400025364370,9_5jw,행복했던 9월 고마워,일상,19,11
4,17841400039750009,bichon.mayo,협찬 동결건조야채블럭 1개로 13종의 보라야채와 유산균 섭취가능 보라색 안토시아닌이...,반려동물,8,211
5,17841400039750009,bichon.mayo,repost royaltailskorea with letrepost 글로리아 리그램...,반려동물,8,572
6,17841400039750009,bichon.mayo,협찬 개과천선 덕스잇 오리 장난감 요즘 핫한 댕난감 너무 귀여워요 약간 내새꾸 iq...,반려동물,8,312
...,...,...,...,...,...,...
39925,17841453615191128,bong_camper83,아빠들의 일사분란함은 가족애 였다 어제 캠핑하는데 전혀 예상치 못했던 비바람 돌풍이...,홈/리빙,25,209
39926,17841453615191128,bong_camper83,광고 식용유 없이 전을 굽는다고 풀무원 철판수제전 3종세트 철판 바삭감자채전 철판 ...,홈/리빙,25,536
39927,17841453615191128,bong_camper83,제품제공 이걸 소개할 수 있어 영광입니다 가장 카고스럽게 보다 더 대담하게 카고컨테...,홈/리빙,25,366
39928,17841453615191128,bong_camper83,제품제공 이제 남편들이 요리 다하겠습니다 아내님들 남편에게 식스볼트 듀라박스만 사주...,홈/리빙,25,342


In [271]:
final_fine_tuning_data = fine_tuning_data.drop(['acnt_id', 'acnt_nm', 'revised_label', 'length'], axis=1)

In [272]:
final_fine_tuning_data.to_csv("final_fine_tuning_data.csv")